In [61]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [62]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [63]:
#links proyectos
def base_inicial_paraguay(link):
    parserT = parser(link)

    paginacion = []

    link = 'https://www.conacyt.gov.py'
    for x in range(5):
        if x == 0:
            paginacion.append(link + '/convocatorias')
        if x != 0:
            paginacion.append(link+ '/convocatorias' + '?page=' + str(x))


    fechas_cierre = []
    estados = []
    urls = []
    tit = []

    for pag in paginacion:
        parserPag = parser(pag)

        titulo = parserPag.xpath('//td[@class="views-field views-field-title views-align-left"]//text()')
        links = parserPag.xpath('//td[@class="views-field views-field-title views-align-left"]//a/@href')
        fecha_cierre = parserPag.xpath('//td[@class="views-field views-field-field-fecha-cierre"]//text()')
        estado = parserPag.xpath('//td[@class="views-field views-field-field-estado-convocatoria"]//text()')

        #Fechas de cierre
        
        for x in fecha_cierre:
            if len(x.strip()) != 0:
                fechas_cierre.append(x)

        #Estado del proyecto
        
        for est in estado:
            estados.append(est.strip())

        #Urls
        
        for l in links:
            urls.append(link+l)

        #Titulos
        
        for x in titulo:
            if len(x.strip()) != 0:
                tit.append(x)

    paraguay = pd.DataFrame()

    paraguay['Título'] = tit
    paraguay['Fecha Cierre'] = fechas_cierre
    paraguay['Estado'] = estados
    paraguay['Link'] = urls

    return paraguay

def descripcion_proyecto(link):

    parserT = parser(link)

    descripcion = parserT.xpath('//div[@class="field-item even"]//text()')

    descrip = ''
    for desc in descripcion:

        descrip = descrip + desc.strip() + ' '

    return descrip


def pdf(link):    
    parserT = parser(link)

    urls = parserT.xpath('//div[@class="field-item even"]//a/@href')

    pdfs = ''
    link = 'https://www.conacyt.gov.py'
    for url in urls:
        if url.endswith('.pdf'):
            pdfs = pdfs + link+ url + ', '

    return pdfs

In [78]:
    
def paraguay():
    link = 'https://www.conacyt.gov.py/convocatorias'

    paraguay = base_inicial_paraguay(link)

    descripciones = []
    pdfs = []

    for link in paraguay['Link']:
        descripciones.append(descripcion_proyecto(link))
        pdfs.append(pdf(link))

    paraguay['Descripción'] = descripciones
    paraguay['Pdfs'] = pdfs

    return paraguay

In [80]:
paraguay.to_excel('Paraguay.xlsx')
    